In [ ]:
# ============================================================
# 1. Install Gradio & Hugging Face dependencies
# ============================================================
!pip install gradio joblib huggingface_hub

import pandas as pd
import joblib
import gradio as gr
from huggingface_hub import hf_hub_download

# ============================================================
# 2. Load model from Hugging Face Hub
# ============================================================
REPO = "<your-username>/<repo-name>"
MODEL_FILE = "random_forest_model.pkl"   # or "xgboost_model.pkl"

model_path = hf_hub_download(repo_id=REPO, filename=MODEL_FILE)
model = joblib.load(model_path)

print("Loaded model:", MODEL_FILE)


# ============================================================
# 3. Load dataset for dropdowns
# ============================================================
df = pd.read_csv("../data/spotify_cleaned_data.csv")


# ============================================================
# 4. Prediction function (Gradio API)
# ============================================================
def predict_popularity_api(energy, danceability, valence, loudness, tempo, explicit, genre):

    sample = pd.DataFrame([{
        "energy": energy,
        "danceability": danceability,
        "valence": valence,
        "loudness": loudness,
        "tempo": tempo,
        "explicit": explicit,
        "macro_genre": genre,
        "artist_popularity": 50,
        "loudness_danceability": loudness * danceability,
        "energy_valence": energy * valence,
        "instrumentalness": 0,
        "acousticness": 0,
        "liveness": 0,
        "speechiness": 0,
        "duration_min": 3
    }])

    prediction = model.predict(sample)[0]
    return float(prediction)


# ============================================================
# 5. Gradio Interface
# ============================================================
interface = gr.Interface(
    fn=predict_popularity_api,
    inputs=[
        gr.Slider(0, 1, step=0.01, label="Energy"),
        gr.Slider(0, 1, step=0.01, label="Danceability"),
        gr.Slider(0, 1, step=0.01, label="Valence"),
        gr.Slider(-60, 0, step=1, label="Loudness"),
        gr.Slider(50, 200, step=1, label="Tempo"),
        gr.Dropdown([True, False], label="Explicit"),
        gr.Dropdown(sorted(df["macro_genre"].unique()), label="Macro Genre")
    ],
    outputs="number",
    title="🎧 Spotify Popularity Predictor (Model from HuggingFace)"
)

interface.launch()
